<a href="https://colab.research.google.com/github/AnberAziz/BS-DS-Project/blob/main/AnberW12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# 🚀 Week 12: Model Deployment (Web App)
# Student: Anber Aziz (Roll No. 2225165090)
# Project: Student Performance Prediction
# ==========================================

import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Install dependencies (Flask & Ngrok for Colab tunneling)
print("Installing Flask and pyngrok...")
!pip install flask pyngrok > /dev/null

print("\n--- Step 1: Saving Models for Deployment ---")
# We assume the models 'model' (ANN) and 'text_clf' (NLP) exist from previous weeks.
# If you restarted the session, we quickly recreate dummy versions for demonstration.

if 'model' not in locals():
    print("⚠️ Previous models not found. Creating dummy models for demo...")
    # Dummy ANN
    model = tf.keras.Sequential([tf.keras.layers.Dense(3, activation='softmax', input_shape=(18,))])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    # Dummy NLP
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.pipeline import Pipeline
    text_clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
    text_clf.fit(["good student", "bad student"], ["High", "Low"])

# Save ANN (Keras format)
model.save('student_ann_model.keras')
print("✅ Saved ANN model as 'student_ann_model.keras'")

# Save NLP Pipeline (Pickle format)
with open('nlp_pipeline.pkl', 'wb') as f:
    pickle.dump(text_clf, f)
print("✅ Saved NLP pipeline as 'nlp_pipeline.pkl'")

print("\n--- Step 2: Creating the Flask App (app.py) ---")

# We write the python code to a file named 'app.py'
app_code = """
from flask import Flask, request, jsonify, render_template_string
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

app = Flask(__name__)

# Load Models
print("Loading models...")
ann_model = tf.keras.models.load_model('student_ann_model.keras')
with open('nlp_pipeline.pkl', 'rb') as f:
    nlp_model = pickle.load(f)
print("Models loaded!")

# HTML Template (The UI)
html_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>Student Performance Predictor</title>
    <style>
        body { font-family: sans-serif; max-width: 600px; margin: auto; padding: 20px; }
        .container { border: 1px solid #ccc; padding: 20px; border-radius: 10px; }
        input, select, textarea { width: 100%; margin-bottom: 10px; padding: 8px; }
        button { background-color: #28a745; color: white; border: none; padding: 10px; cursor: pointer; width: 100%; }
        button:hover { background-color: #218838; }
        .result { margin-top: 20px; font-weight: bold; }
    </style>
</head>
<body>
    <div class="container">
        <h2>🎓 Student Performance AI</h2>

        <h3>1. Numeric Data (ANN)</h3>
        <label>Raised Hands (0-100):</label>
        <input type="number" id="raised_hands" value="50">

        <label>Visited Resources (0-100):</label>
        <input type="number" id="visited_resources" value="50">

        <label>Absence Days:</label>
        <select id="absence">
            <option value="0">Under 7 Days</option>
            <option value="1">Above 7 Days</option>
        </select>

        <h3>2. Teacher Comments (NLP)</h3>
        <label>Enter feedback:</label>
        <textarea id="comment" rows="3">Student is attentive and submits work on time.</textarea>

        <button onclick="predict()">Predict Performance</button>

        <div class="result" id="output"></div>
    </div>

    <script>
        async function predict() {
            const hands = document.getElementById('raised_hands').value;
            const resources = document.getElementById('visited_resources').value;
            const absence = document.getElementById('absence').value;
            const comment = document.getElementById('comment').value;

            const response = await fetch('/predict', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({
                    features: [hands, resources, absence],
                    text: comment
                })
            });

            const data = await response.json();
            document.getElementById('output').innerHTML =
                `Numeric Prediction: ${data.ann_prediction} <br> Text Prediction: ${data.nlp_prediction}`;
        }
    </script>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(html_template)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json

    # 1. NLP Prediction
    text_pred = nlp_model.predict([data['text']])[0]

    # 2. ANN Prediction (Simplified for demo - normally needs full preprocessing)
    # Ideally, we would load the scaler here too. For this demo, we mock the input shape.
    # In a real app, you must match the exact 18 input features of X_train.
    # Here we return a dummy response if input shape doesn't match, just to show the connection works.
    try:
        # Dummy prediction logic for demo (since we can't easily recreate the 18-feature vector manually in UI)
        ann_pred = "Calculated (Mock)"
    except:
        ann_pred = "Error"

    return jsonify({'ann_prediction': ann_pred, 'nlp_prediction': text_pred})

if __name__ == '__main__':
    app.run(port=5000)
"""

with open('app.py', 'w') as f:
    f.write(app_code)
print("✅ Created app.py")

print("\n--- Step 3: Launching Server ---")
print("1. Copy the Authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken")
print("2. Paste it below (or just press Enter if you have done this before, though it might fail without a token).")

# Ask user for token (Safety first!)
token = input("Enter Ngrok Authtoken (or press Enter to try without): ")
if token:
    !ngrok config add-authtoken $token

from pyngrok import ngrok

# Terminate open tunnels
ngrok.kill()

# Open tunnel
public_url = ngrok.connect(5000)
print(f"\n🚀 YOUR PUBLIC URL: {public_url}")
print("Click the link above to see your Web App!")

# Run Flask in background
!nohup python app.py &

Installing Flask and pyngrok...

--- Step 1: Saving Models for Deployment ---
⚠️ Previous models not found. Creating dummy models for demo...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Saved ANN model as 'student_ann_model.keras'
✅ Saved NLP pipeline as 'nlp_pipeline.pkl'

--- Step 2: Creating the Flask App (app.py) ---
✅ Created app.py

--- Step 3: Launching Server ---
1. Copy the Authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken
2. Paste it below (or just press Enter if you have done this before, though it might fail without a token).
Enter Ngrok Authtoken (or press Enter to try without): 37Z2f8c9T5e48P6xvGcLNaBaZDH_2E2E89wukkwH9yRqgz9d5
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml

🚀 YOUR PUBLIC URL: NgrokTunnel: "https://collaborative-kimbery-unchangefully.ngrok-free.dev" -> "http://localhost:5000"
Click the link above to see your Web App!
nohup: appending output to 'nohup.out'
